In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
def read_gt(folder_path,file_name):
    os.chdir(folder_path)
    data = open(file_name,'r').read()
    return data

In [4]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [5]:
def convert_to_df(data):
    data_split1 = data.split("\n")
    num_data_split = []
    for row in data_split1:
        row_split = row.split(",")
        num_row_split = []
        for ele in row_split:
            if isfloat(ele):
                num_row_split.append(float(ele))
            else:
                num_row_split.append(np.NAN)
        num_data_split.append(num_row_split)
    data_df = pd.DataFrame(num_data_split)
    return data_df

In [6]:
def format_df(initial_df,reso_x,reso_y):
    np_matrix = initial_df.values
    x = ((np_matrix[:,0]/reso_x)+(np_matrix[:,2]/reso_x))/2
    y = ((np_matrix[:,1]/reso_y)+(np_matrix[:,3]/reso_y))/2
    w = -((np_matrix[:,0]/reso_x)-(np_matrix[:,2]/reso_x))
    h = -((np_matrix[:,1]/reso_y)-(np_matrix[:,3]/reso_y))
    final_np_matrix = np.column_stack((x,y,w,h))
    final_df = pd.DataFrame(final_np_matrix,columns = ['mid_x','mid_y','width','height'])
    return final_df

In [7]:
def convert_txt_to_df(folder_path,file_name,reso_x,reso_y):
    data = read_gt(folder_path, file_name)
    data_df = convert_to_df(data)
    final_df = format_df(data_df,reso_x,reso_y)
    return final_df

In [8]:
def convert_to_rnn_format(object_class,gt_df):
    final_df = pd.DataFrame()
    labels = ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "street sign", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "hat", "backpack", "umbrella", "shoe", "eye", "glasses", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "plate", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "mirror", "dining table", "window", "desk", "toilet", "door", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "blender", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush", "hair brush"] 
    for index,row in gt_df.iterrows():
        random_3d = np.random.randn(19,19,96)
        random_3d = random_3d - random_3d.astype(int)
        random_3d = np.absolute(random_3d)
        random_3d[:,:,0] = np.zeros((19,19))
        xcenter = row['mid_x']
        ycenter = row['mid_y']
        if np.isnan(xcenter) == False and np.isnan(ycenter) == False:
            xgrid_num = int(xcenter/(1/19))
            xcenter_wrt_grid = xcenter/(1/19) - int(xcenter/(1/19))
            ygrid_num = int(ycenter/(1/19))
            if xgrid_num<1 and ygrid_num<1:    
                ycenter_wrt_grid = ycenter/(1/19) - int(ycenter/(1/19))
                random_3d[xgrid_num,ygrid_num,0] = 1
                random_3d[xgrid_num,ygrid_num,1] = xcenter_wrt_grid
                random_3d[xgrid_num,ygrid_num,2] = xcenter_wrt_grid
                random_3d[xgrid_num,ygrid_num,3] = row['height']
                random_3d[xgrid_num,ygrid_num,4] = row['width']
                random_3d[xgrid_num,ygrid_num,5:96] = 0
                thing_num = 0
                for thing in labels:
                    if object_class == thing:
                        random_3d[xgrid_num][ygrid_num][thing_num+5] = 1
                        break
                    thing_num = thing_num+1
        final_flat = random_3d.flatten()
        ser = pd.Series(final_flat)
        final_df = final_df.append(ser,ignore_index = True)
    return final_df

Rest of the code is just to make csv files for all the text ground_truths we have

In [8]:
Babenko_girl = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Babenko/girl","gt.txt",320,240)
Babenko_girl = convert_to_rnn_format('person',Babenko_girl)
Babenko_girl.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Babenko/girl/rnn_gt.csv",sep='\t', encoding='utf-8')

In [9]:
Babenko_occludedface = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Babenko/OccludedFace2","gt.txt",320,240)
Babenko_occludedface =convert_to_rnn_format('person',Babenko_occludedface)
Babenko_occludedface.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Babenko/OccludedFace2/rnn_gt.csv",sep='\t', encoding='utf-8')

In [10]:
Babenko_surfer = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Babenko/surfer","gt.txt",480,360)
Babenko_surfer = convert_to_rnn_format('person',Babenko_surfer)
Babenko_surfer.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Babenko/surfer/rnn_gt.csv",sep='\t', encoding='utf-8')

In [11]:
bobot_vida_ball = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_A_ball","gt.txt",320,240)
bobot_vida_ball = convert_to_rnn_format('sports ball',bobot_vida_ball)
bobot_vida_ball.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_A_ball/rnn_gt.csv",sep='\t', encoding='utf-8')

In [12]:
bobot_vidb = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_B_cup","gt.txt",320,240)
bobot_vidb = convert_to_rnn_format("cup",bobot_vidb)
bobot_vidb.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_B_cup/rnn_gt.csv",sep='\t', encoding='utf-8')

In [13]:
bobot_vidc = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_C_juice","gt.txt",320,240)
bobot_vidc = convert_to_rnn_format("dining table",bobot_vidc)
bobot_vidc.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_C_juice/rnn_gt.csv",sep='\t', encoding='utf-8')

In [14]:
bobot_vidd = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_D_person","gt.txt",320,240)
bobot_vidd = convert_to_rnn_format("person",bobot_vidd)
bobot_vidd.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_D_person/formatted_gt.csv",sep='\t', encoding='utf-8')

In [15]:
bobot_vide = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_E_person_part_occluded","gt.txt",320,240)
bobot_vide = convert_to_rnn_format("person",bobot_vide)
bobot_vide.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_E_person_part_occluded/rnn_gt.csv",sep='\t', encoding='utf-8')

In [16]:
bobot_vide = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_E_person_part_occluded","gt.txt",320,240)
bobot_vide = convert_to_rnn_format("person",bobot_vide)
bobot_vide.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_E_person_part_occluded/rnn_gt.csv",sep='\t', encoding='utf-8')

In [17]:
bobot_vidf = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_F_person_fully_occluded","gt.txt",320,240)
bobot_vidf = convert_to_rnn_format("person",bobot_vidf)
bobot_vidf.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_F_person_fully_occluded/rnn_gt.csv",sep='\t', encoding='utf-8')

In [18]:
bobot_vidg = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_G_person_fully_occluded","gt.txt",320,240)
bobot_vidg = convert_to_rnn_format("person",bobot_vidg)
bobot_vidg.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_G_person_fully_occluded/rnn_gt.csv",sep='\t', encoding='utf-8')


In [19]:
bobot_vidh = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_H_panda","gt.txt",320,240)
bobot_vidh = convert_to_rnn_format("teddy bear",bobot_vidh)
bobot_vidh.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_H_panda/rnn_gt.csv",sep='\t', encoding='utf-8')


In [20]:
bobot_vidi = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_I_person_crossing","gt.txt",320,240)
bobot_vidi = convert_to_rnn_format("person",bobot_vidi)
bobot_vidi.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_I_person_crossing/rnn_gt.csv",sep='\t', encoding='utf-8')

In [21]:
bobot_vidj = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_J_person_floor","gt.txt",320,240)
bobot_vidj = convert_to_rnn_format("person",bobot_vidj)
bobot_vidj.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_J_person_floor/rnn_gt.csv",sep='\t', encoding='utf-8')

In [22]:
bobot_vidk = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_K_cup","gt.txt",320,240)
bobot_vidk = convert_to_rnn_format("cup",bobot_vidk)
bobot_vidk.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_K_cup/rnn_gt.csv",sep='\t', encoding='utf-8')

In [23]:
bobot_vidl = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_L_coffee","gt.txt",320,240)
bobot_vidl = convert_to_rnn_format("cup",bobot_vidl)
bobot_vidl.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/BoBot/Vid_L_coffee/rnn_gt.csv",sep='\t', encoding='utf-8')

In [24]:
cehovin1 = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Cehovin/dinosaur","gt.txt",320,240)
cehovin1 = convert_to_rnn_format("teddy bear",cehovin1)
cehovin1.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Cehovin/dinosaur/rnn_gt.csv",sep='\t', encoding='utf-8')

In [25]:
cehovin2 = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Cehovin/gymnastics","gt.txt",320,180)
cehovin2 = convert_to_rnn_format("person",cehovin2)
cehovin2.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Cehovin/gymnastics/rnn_gt.csv",sep='\t', encoding='utf-8')

In [26]:
cehovin3 = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Cehovin/hand","gt.txt",320,240)
cehovin3 = convert_to_rnn_format("person",cehovin3)
cehovin3.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Cehovin/hand/rnn_gt.csv",sep='\t', encoding='utf-8')

In [27]:
cehovin4 = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Cehovin/hand2","gt.txt",320,240)
cehovin4 = convert_to_rnn_format("person",cehovin4)
cehovin4.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Cehovin/hand2/rnn_gt.csv",sep='\t', encoding='utf-8')

In [28]:
cehovin5 = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Cehovin/torus","gt.txt",320,240)
cehovin5 = convert_to_rnn_format("donut",cehovin5)
cehovin5.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Cehovin/torus/rnn_gt.csv",sep='\t', encoding='utf-8')

In [29]:
Ellis_ijcv2011_1 = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ellis_ijcv2011/head_motion","gt.txt",320,240)
Ellis_ijcv2011_1 = convert_to_rnn_format("person",Ellis_ijcv2011_1)
Ellis_ijcv2011_1.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ellis_ijcv2011/head_motion/rnn_gt.csv",sep='\t', encoding='utf-8')

In [30]:
Ellis_ijcv2011_2 = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ellis_ijcv2011/shaking_camera","gt.txt",320,240)
Ellis_ijcv2011_2 = convert_to_rnn_format("desk",Ellis_ijcv2011_2)
Ellis_ijcv2011_2.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ellis_ijcv2011/shaking_camera/rnn_gt.csv",sep='\t', encoding='utf-8')

In [31]:
Ellis_ijcv2011_3 = convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ellis_ijcv2011/track_running","gt.txt",768,576)
Ellis_ijcv2011_3 = convert_to_rnn_format("person",Ellis_ijcv2011_3)
Ellis_ijcv2011_3.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ellis_ijcv2011/track_running/rnn_gt.csv",sep='\t', encoding='utf-8')

In [32]:
Godec1= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/cliff-dive1","gt.txt",400,226)
Godec1 = convert_to_rnn_format("person",Godec1)
Godec1.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/cliff-dive1/rnn_gt.csv",sep='\t', encoding='utf-8')

In [33]:
Godec2= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/cliff-dive2","gt.txt",400,226)
Godec2 = convert_to_rnn_format("person",Godec2)
Godec2.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/cliff-dive2/rnn_gt.csv",sep='\t', encoding='utf-8')

In [34]:
Godec3= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/motocross1","gt.txt",640,360)
Godec3 = convert_to_rnn_format("motorcycle",Godec3)
Godec3.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/motocross1/rnn_gt.csv",sep='\t', encoding='utf-8')

In [35]:
Godec4= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/motocross2","gt.txt",640,360)
Godec4 = convert_to_rnn_format("motorcycle",Godec4)
Godec4.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/motocross2/rnn_gt.csv",sep='\t', encoding='utf-8')

In [36]:
Godec5= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/mountain-bike","gt.txt",640,360)
Godec5 = convert_to_rnn_format("bicycle",Godec5)
Godec5.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/mountain-bike/rnn_gt.csv",sep='\t', encoding='utf-8')

In [37]:
Godec6= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/skiing","gt.txt",640,360)
Godec6 = convert_to_rnn_format("person",Godec6)
Godec6.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/skiing/rnn_gt.csv",sep='\t', encoding='utf-8')

In [38]:
Godec7= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/volleyball","gt.txt",720,576)
Godec7 = convert_to_rnn_format("sports ball",Godec7)
Godec7.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Godec/volleyball/rnn_gt.csv",sep='\t', encoding='utf-8')

In [39]:
Kalal1= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/car","gt.txt",320,240)
Kalal1 = convert_to_rnn_format("car",Kalal1)
Kalal1.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/car/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
Kalal2= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/CarChase","gt.txt",290,217)
Kalal2 = convert_to_rnn_format("car",Kalal2)
Kalal2.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/CarChase/rnn_gt.csv",sep='\t', encoding='utf-8')

In [43]:
Kalal3= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/david","gt.txt",320,240)
Kalal3 = convert_to_rnn_format("person",Kalal3)
Kalal3.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/david/rnn_gt.csv",sep='\t', encoding='utf-8')

In [44]:
Kalal4= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/jumping","gt.txt",352,288)
Kalal4 = convert_to_rnn_format("person",Kalal4)
Kalal4.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/jumping/rnn_gt.csv",sep='\t', encoding='utf-8')

In [45]:
Kalal5= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/Motocross","gt.txt",470,310)
Kalal5 = convert_to_rnn_format("motorcycle",Kalal5)
Kalal5.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/Motocross/rnn_gt.csv",sep='\t', encoding='utf-8')

In [46]:
Kalal6= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/Panda","gt.txt",312,233)
Kalal6 = convert_to_rnn_format("bear",Kalal6)
Kalal6.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/Panda/rnn_gt.csv",sep='\t', encoding='utf-8')

In [47]:
Kalal7= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/pedestrian3","gt.txt",320,240)
Kalal7 = convert_to_rnn_format("person",Kalal7)
Kalal7.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/pedestrian3/rnn_gt.csv",sep='\t', encoding='utf-8')

In [50]:
Kalal8= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/pedestrian4","gt.txt",320,240)
Kalal8 = convert_to_rnn_format("person",Kalal8)
Kalal8.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/pedestrian4/rnn_gt.csv",sep='\t', encoding='utf-8')

In [51]:
Kalal9= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/pedestrian5","gt.txt",320,240)
Kalal9 = convert_to_rnn_format("person",Kalal9)
Kalal9.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/pedestrian5/rnn_gt.csv",sep='\t', encoding='utf-8')

In [52]:
Kalal10= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/Volkswagen","gt.txt",640,480)
Kalal10 = convert_to_rnn_format("truck",Kalal10)
Kalal10.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kalal/Volkswagen/rnn_gt.csv",sep='\t', encoding='utf-8')

KeyboardInterrupt: 

In [ ]:
Kwon1= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kwon/diving","gt.txt",400,224)
Kwon1 = convert_to_rnn_format("person",Kwon1)
Kwon1.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kwon/diving/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
Kwon2= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kwon/gym","gt.txt",426,234)
Kwon2 = convert_to_rnn_format("person",Kwon2)
Kwon2.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kwon/gym/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
Kwon3= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kwon/jump","gt.txt",416,234)
Kwon3 = convert_to_rnn_format("person",Kwon3)
Kwon3.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kwon/jump/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
Kwon4= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kwon/trans","gt.txt",640,332)
Kwon4 = convert_to_rnn_format("person",Kwon4)
Kwon4.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Kwon/trans/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
Other1= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Other/Asada","gt.txt",480,270)
Other1 = convert_to_rnn_format("person",Other1)
Other1.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Other/Asada/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
Other2= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Other/drunk2","gt.txt",508,336)
Other2 = convert_to_rnn_format("car",Other2)
Other2.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Other/drunk2/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
Other3= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Other/dudek-face","gt.txt",720,448)
Other3 = convert_to_rnn_format("person",Other3)
Other3.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Other/dudek-face/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
Other4= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Other/faceocc1","gt.txt",352,288)
Other4 = convert_to_rnn_format("person",Other4)
Other4.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Other/faceocc1/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
Other5= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Other/figure_skating","gt.txt",480,270)
Other5 = convert_to_rnn_format("person",Other5)
Other5.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Other/figure_skating/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
Other6= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Other/woman","gt.txt",352,288)
Other6 = convert_to_rnn_format("person",Other6)
Other6.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Other/woman/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
prost1= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/PROST/board","gt.txt",640,480)
prost1 = convert_to_rnn_format("remote",prost1)
prost1.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/PROST/board/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
prost2= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/PROST/box","gt.txt",640,480)
prost2 = convert_to_rnn_format("box",prost2)
prost2.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/PROST/box/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
prost3= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/PROST/lemming","gt.txt",640,480)
prost3 = convert_to_rnn_format("teddy bear",prost3)
prost3.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/PROST/lemming/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
prost4= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/PROST/liquor","gt.txt",640,480)
prost4 = convert_to_rnn_format("bottle",prost4)
prost4.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/PROST/liquor/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
ross1= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ross/car11","gt.txt",320,240)
ross1 = convert_to_rnn_format("car",ross1)
ross1.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ross/car11/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
ross2= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ross/dog1","gt.txt",320,240)
ross2 = convert_to_rnn_format("dog",ross2)
ross2.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ross/dog1/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
ross3= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ross/Sylvestr","gt.txt",320,240)
ross3= convert_to_rnn_format("teddy bear",ross3)
ross3.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ross/Sylvestr/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
ross4= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ross/trellis","gt.txt",320,240)
ross4 = convert_to_rnn_format("person",ross4)
ross4.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Ross/trellis/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
thang1= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Thang/coke","gt.txt",320,240)
thang1= convert_to_rnn_format("cup",thang1)
thang1.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Thang/coke/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
thang2= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Thang/person","gt.txt",320,240)
thang2 = convert_to_rnn_format("person",thang2)
thang2.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Thang/person/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
thang3= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Thang/tiger1","gt.txt",320,240)
thang3 = convert_to_rnn_format("teddy bear",thang3)
thang3.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Thang/tiger1/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
thang4= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Thang/tiger2","gt.txt",320,240)
thang4 = convert_to_rnn_format("teddy bear",thang4)
thang4.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Thang/tiger2/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
wang1= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Wang/bird_1","gt.txt",720,400)
wang1= convert_to_rnn_format("bird",wang1)
wang1.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Wang/bird_1/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
wang2= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Wang/bird_2","gt.txt",720,400)
wang2 = convert_to_rnn_format("bird",wang2)
wang2.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Wang/bird_2/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
wang3= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Wang/bolt","gt.txt",640,360)
wang3 = convert_to_rnn_format("bird",wang3)
wang3.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Wang/bolt/rnn_gt.csv",sep='\t', encoding='utf-8')

In [ ]:
wang4= convert_txt_to_df("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Wang/girl_mov","gt.txt",640,480)
wang4 = convert_to_rnn_format("person",wang4)
wang4.to_csv("/home/soham/AeroMIT/dataset_rnn_yolo/TrackingDataset/Wang/girl_mov/rnn_gt.csv",sep='\t', encoding='utf-8')